In [5]:
import os
import yaml
import json
import argparse
import threading
import numpy as np
from datetime import datetime
from typing import List
from langchain.prompts import PromptTemplate
from asn.env.environment import Environment
from asn.data.data import Data
from asn.agent.agent import Agent, LLMAgent
from asn.agent.action import Act
from asn.llm.llm import LLMManager
from asn.llm.prompt import PROMPT_EVALUATE_ACT, PROMPT_EVALUATE_POST, PROMPT_EMOTION_CLASSIFICATION
from asn.utils.time import *
from asn.utils.logger import get_logger, set_logger

import warnings
warnings.filterwarnings("ignore")

# set_logger(log_folder="log_eval")
# args = argparse.ArgumentParser()
# args.add_argument("--config_path", "-c", type=str, default="config/example.yaml")
# args.add_argument("--data_path", "-d", type=str, default="dataset/example.json")
# args = args.parse_args()
args = argparse.Namespace(config_path="config/example.yaml", data_path="dataset/example.json")
conf_path = args.config_path
with open(conf_path, "r") as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)
for key, value in args.__dict__.items():
    conf[key] = value

# print settings
settings = "Settings:\n"
for key, value in conf.items():
    settings += f"{key}: {value}\n"
get_logger().info(settings)
get_logger().debug(settings)
print(settings)

# Random seed
if "seed" in conf:
    seed = conf["seed"]
else:
    seed = 0
import random
random.seed(conf["seed"])
import numpy as np
np.random.seed(conf["seed"])

# Set LLMManager
LLMManager.set_manager(conf["llm"])

# Load data and environment
save_path = conf["save_path"] + f"/model_{conf['time_sim_end'].replace(' ', '_')}"
data = Data.load_from_data(conf["data_path"])
with open(os.path.join(save_path, f"model.json"), "r") as f:
    model_dict = json.load(f)
env = Environment.load_from_dict(model_dict["env"])

for user in env.users:
    user.mastodon_info = data.get_meta(user.id, "mastodon_info")


Settings:
seed: 202505
llm: {'llm_name': 'OpenAI', 'llm_model': 'qwen-plus', 'llm_url': 'https://dashscope.aliyuncs.com/compatible-mode/v1', 'embed_name': 'OpenAI', 'embed_model': '/data1/zhushengmao/gte_Qwen2-7B-instruct', 'embed_url': 'http://localhost:8002/v1', 'api_key': 'sk-2b26e585f3aa4c17b949f2e675a5284e'}
topic: example
data_path_raw: dataset_raw/example.json
data_path: dataset/example.json
num_users: 20
sampling_strategy: active_by_time
react_strategy: batch
simulate_with_data: False
interval: 30M
time_intv: 24H
time_init_begin: 2024-02-01 00:00:00
time_init_end: 2024-02-11 00:00:00
time_sim_begin: 2024-02-11 00:00:00
time_sim_end: 2024-02-12 00:00:00
load_model: /data1/zhushengmao/save/example/model_init/model.json
save_path: /data1/zhushengmao/save/example
time_window_size: 12H
active_window_size: 3H
neg_sample_num: 1
max_thread_num: 1
max_workers: 50
parallel: True
debug: False
config_path: config/example.yaml

LLM Manager set to: OpenAI, qwen-plus, OpenAI, <asn.llm.llm.Ope

In [6]:
print(len(env.log))

1839


In [3]:
import threading
from mastodon import Mastodon
from asn.utils.mastodon import *
from asn.llm.prompt import Prompts
from multiavatar.multiavatar import multiavatar
from cairosvg import svg2png
import time


# 统一env.logs转发的type
for log in env.log:
    if log["act"]["type"] == "retweet" or log["act"]["type"] == "repost" or log["act"]["type"] == "share":
        log["act"]["type"] = "repost"

# 翻译用户profile
# print("翻译用户profile")
# def translate_profile(user):
#     llm = LLMManager.get_llm()
#     characteristics = user.agent.profile.characteristics
#     characteristics = characteristics[characteristics.find("characteristics:")+len("characteristics:"):]
#     prompt = Prompts.post_translation.format(post_text=characteristics)
#     user.agent.profile.characteristics_translated = llm._call(prompt)
# threads = []
# for user in env.users:
#     t = threading.Thread(target=translate_profile, args=(user,))
#     threads.append(t)
#     t.start()
# for t in threads:
#     t.join()
for user in env.users:
    print(user.agent.profile.characteristics_translated)

# 登录每一个用户
for user in env.users:
    user.access_token = user.mastodon_info["access_token"]
    user.mastodon = Mastodon(
        access_token = user.mastodon_info["access_token"],
        api_base_url = user.mastodon_info["api_base_url"]
    )
    mastodon = Mastodon(
        access_token = user.mastodon_info["access_token"],
        api_base_url = user.mastodon_info["api_base_url"]
    )

# 为每一个用户更新profile
for user in env.users:
    # note
    user.mastodon.account_update_credentials(note=user.agent.profile.characteristics_translated)

# 翻译帖子
print("翻译帖子")
def translate_post(message):
    llm = LLMManager.get_llm()
    prompt = Prompts.post_translation.format(post_text=message.text)
    if message.text_translated is not None:
        return
    try:
        message.text_translated = llm._call(prompt)
    except Exception as e:
        message.text_translated = message.text
threads = []
for message in env.messages:
    t = threading.Thread(target=translate_post, args=(message,))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

# 保存model
save_path = conf["save_path"] + f"/model_{conf['time_sim_end'].replace(' ', '_')}"
with open(os.path.join(save_path, f"model.json"), "w") as f:
    model_dict = {
        "env": env.save_to_dict(save_path)
    }
    json.dump(model_dict, f, indent=4)

time_sim_begin = conf["time_init_begin"]
time_sim_end = conf["time_sim_end"]
time_intv = conf["interval"]
time_step = time_sim_begin
cnt_log = 0

经常为乌克兰发声，频繁分享与俄乌冲突相关的想法和经历。你的活跃程度中等到高，专注于撰写详细的推文并参与那些突出乌克兰视角、战争罪行和地缘政治问题的帖子。你对那些被视为背叛或削弱乌克兰斗争的人表达出强烈的情感，通常是挫败感和愤怒。你的互动显示出对乌克兰身份的深厚自豪感，并致力于提高人们对战争现实的认识。正义、真相和团结等价值观是你帖子的核心，偶尔也会触及个人在动力和疲惫方面的挣扎。 #UkrainianView
一个热心且活跃的志愿者，经常发布关于支持乌克兰士兵的慈善活动的更新。您的重点是创造和分发 essential items 像 trench candles 和 balms，展示生产过程并表达对捐款的感激之情。您强调社区参与，通过详细的报告和照片突出贡献的影响。您的价值观反映了强烈的爱国主义和同情心，因为您一贯倡导以温暖和关怀支持乌克兰军队。尽管偶尔有不活跃的时期，但您的帖子显示了一个充满奉献精神和民族自豪感的、有组织的个人。
有意识的用户，与乌克兰观点和全球政治有着强烈的联系，特别是关注与俄罗斯的冲突。你的活跃程度适中，主要围绕点赞，偶尔撰写帖子，突出乌克兰的挣扎和韧性。你表达出对人权的深切关注，反对压迫政权，并希望提高国际社会的认知和支持。你的互动显示出致力于传播真实信息和对抗宣传，常常使用讽刺或沮丧来传达局势的严重性。尽管有时感到疲惫和绝望，你的帖子反映出对正义和#UkrainianView社区团结的希望。
频繁但有目的性的社交媒体用户，主要与乌克兰身份认同及支持与民族自豪感相符的事业相关的内容互动。你最近的活动包括发起并推广一项重要的无人机筹款活动，展现出对支持乌克兰防御努力的坚定承诺。通过分享和点赞有关文化遗产保护和抵制身份抹除的帖子，你表达了与保护乌克兰遗产紧密相连的深厚价值观。尽管你的发帖频率较低，但你的互动突显了你在关乎社区支持和民族团结问题上的热情和积极态度。这表明你更重视有意义的参与而非随意的互动。#UkrainianView @username
对地缘政治问题有浓厚兴趣的媒体用户，特别是与乌克兰和俄罗斯相关的问题。你的活跃程度适中，主要集中在撰写帖子和点赞与你对当前事件和政治观点一致的内容。你清楚地表达了自己的价值观和信仰，经常批评俄罗斯领导人的行为，同时支持乌克兰的立场。正义、历史背景和政治评论的主题主导了你的互动，展示了一个从#Ukra

In [4]:
# 每次发生异常后，从上次的log开始继续
logs = env.log[cnt_log:]
# 遍历&处理每一条log， 每一百条log睡眠一次
while time_step < time_sim_end:
    time_next = add_interval(time_step, time_intv)
    log_step = [log for log in logs if time_step <= log["timestamp"] < time_next]
    print(f"Time: {time_step}")
    for log in log_step:
        user = env.get_user_by_id(log["user"])
        message = env.get_message_by_id(log["message"])
        message = env.get_message_by_id(message.origin_id())
        act = Act.load_from_dict(log["act"])
        if act.type != "read": # 过滤掉read的log
            print(f"User: {user.id}, Act: {act.type}, Message: {message.text_translated}")
        if act.type == "post":
            # 在time_step和time_next之间随机一个时间
            time_post = generate_random_between(time_step, time_next)
            status = post_status_with_time(user.access_token, message.text_translated, time_post)
            message.mastodon_info = {
                "id": status["id"],
                "content": status["content"]
            }
        elif act.type == "like":
            user.mastodon.status_favourite(message.mastodon_info["id"])
        elif act.type == "repost":
            user.mastodon.status_reblog(message.mastodon_info["id"])
        cnt_log += 1
        if cnt_log % 100 == 0:
            time.sleep(10)
    time_step = time_next

Time: 2024-02-01 00:00:00
User: 0, Act: post, Message: 凌晨1点。我尝试写另一个 earlier 承诺过的帖子已经超过一个小时了，但我没有做到。到现在，这更像是在强迫自己。原因有几个。首先，因为真的没有精力去做。 1/19

#UkrainianView
User: 13, Act: like, Message: 凌晨1点。我尝试写另一个 earlier 承诺过的帖子已经超过一个小时了，但我没有做到。到现在，这更像是在强迫自己。原因有几个。首先，因为真的没有精力去做。 1/19

#UkrainianView
User: 18, Act: like, Message: 凌晨1点。我尝试写另一个 earlier 承诺过的帖子已经超过一个小时了，但我没有做到。到现在，这更像是在强迫自己。原因有几个。首先，因为真的没有精力去做。 1/19

#UkrainianView
Time: 2024-02-01 00:30:00
Time: 2024-02-01 01:00:00
Time: 2024-02-01 01:30:00
Time: 2024-02-01 02:00:00
Time: 2024-02-01 02:30:00
Time: 2024-02-01 03:00:00
Time: 2024-02-01 03:30:00
Time: 2024-02-01 04:00:00
Time: 2024-02-01 04:30:00
Time: 2024-02-01 05:00:00
Time: 2024-02-01 05:30:00
Time: 2024-02-01 06:00:00
Time: 2024-02-01 06:30:00
Time: 2024-02-01 07:00:00
Time: 2024-02-01 07:30:00
Time: 2024-02-01 08:00:00
Time: 2024-02-01 08:30:00
Time: 2024-02-01 09:00:00
Time: 2024-02-01 09:30:00
Time: 2024-02-01 10:00:00
Time: 2024-02-01 10:30:00
Time: 2024-02-01 11:00:00
Time: 2024-02

In [8]:
msgid2likes = {msg.id: set() for msg in env.messages}
msgid2retweets = {msg.id: set() for msg in env.messages}
msgid2reads = {msg.id: set() for msg in env.messages}
# 遍历&处理每一条log
time_sim_begin = conf["time_sim_begin"]
time_sim_end = conf["time_sim_end"]
time_intv = conf["interval"]
time_step = time_sim_begin
while time_step < time_sim_end:
    time_next = add_interval(time_step, time_intv)
    log_step = [log for log in env.log if time_step <= log["timestamp"] < time_next]
    print(f"Time: {time_step}")
    for log in log_step:
        user = env.get_user_by_id(log["user"])
        message = env.get_message_by_id(log["message"])
        message = env.get_message_by_id(message.origin_id())
        act = Act.load_from_dict(log["act"])
        if act.type != "read": # 过滤掉read的log
            print(f"User: {user.id}, Act: {act.type}, Message: {message.text_translated}")
        if act.type == "read":
            msgid2reads[message.id].add(user.id)
        if act.type == "like":
            msgid2likes[message.id].add(user.id)
        elif act.type == "repost":
            msgid2retweets[message.id].add(user.id)
    time_step = time_next

    # 保存mid2likes和mid2retweets在文件中
    with open("test.txt", "w") as f:
        for message in env.messages:
            if message.type == "post":
                print(f"Time: {message.timestamp}. Reads: {len(msgid2reads[message.id])}. Likes: {len(msgid2likes[message.id])},{message.liked_by}. Retweets: {len(msgid2retweets[message.id])},{message.reposted_by}. Message: {message.text}", file=f)
            else:
                print(message.type)


Time: 2023-08-11 00:00:00
Time: 2023-08-11 01:00:00
Time: 2023-08-11 02:00:00
Time: 2023-08-11 03:00:00
Time: 2023-08-11 04:00:00
Time: 2023-08-11 05:00:00
Time: 2023-08-11 06:00:00
Time: 2023-08-11 07:00:00
Time: 2023-08-11 08:00:00
Time: 2023-08-11 09:00:00
Time: 2023-08-11 10:00:00
Time: 2023-08-11 11:00:00
Time: 2023-08-11 12:00:00
Time: 2023-08-11 13:00:00
Time: 2023-08-11 14:00:00
Time: 2023-08-11 15:00:00
Time: 2023-08-11 16:00:00
Time: 2023-08-11 17:00:00
Time: 2023-08-11 18:00:00
Time: 2023-08-11 19:00:00
Time: 2023-08-11 20:00:00
Time: 2023-08-11 21:00:00
Time: 2023-08-11 22:00:00
Time: 2023-08-11 23:00:00
Time: 2023-08-12 00:00:00
Time: 2023-08-12 01:00:00
Time: 2023-08-12 02:00:00
Time: 2023-08-12 03:00:00
Time: 2023-08-12 04:00:00
Time: 2023-08-12 05:00:00
Time: 2023-08-12 06:00:00
Time: 2023-08-12 07:00:00
Time: 2023-08-12 08:00:00
Time: 2023-08-12 09:00:00
Time: 2023-08-12 10:00:00
Time: 2023-08-12 11:00:00
Time: 2023-08-12 12:00:00
Time: 2023-08-12 13:00:00
Time: 2023-0

In [7]:
print(len([log for log in env.log if log["act"]["type"] == "post" or log["act"]["type"] == "repost"]))
print(len([log for log in env.log if log["act"]["type"] == "like"]))
time_sim_begin = conf["time_init_begin"]
time_sim_end = conf["time_sim_end"]
time_intv = conf["time_intv"]
time_step = time_sim_begin
while time_step < time_sim_end:
    time_next = add_interval(time_step, time_intv)
    log_step = [log for log in env.log if time_step <= log["timestamp"] < time_next]
    print(f"Time: {time_step}")
    print(len([log for log in log_step if log["act"]["type"] == "read"]))
    print(len([log for log in log_step if log["act"]["type"] == "like"]))
    print(len([log for log in log_step if log["act"]["type"] == "repost"]))
    print(len([log for log in log_step if log["act"]["type"] == "post"]))
    time_step = time_next

66
847
Time: 2024-02-01 00:00:00
0
2
0
3
Time: 2024-02-02 00:00:00
0
5
0
9
Time: 2024-02-03 00:00:00
0
0
0
1
Time: 2024-02-04 00:00:00
0
4
0
3
Time: 2024-02-05 00:00:00
0
5
0
7
Time: 2024-02-06 00:00:00
0
13
0
5
Time: 2024-02-07 00:00:00
0
25
1
15
Time: 2024-02-08 00:00:00
0
7
0
5
Time: 2024-02-09 00:00:00
0
12
1
7
Time: 2024-02-10 00:00:00
0
7
1
8
Time: 2024-02-11 00:00:00
871
767
0
0
